In [1]:
from newspaper import Article
import nltk
import newspaper
import pandas as pd

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import nltk

In [3]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string

# Read Data

In [4]:
header = ["GlobalEventId", "sqldate", "MonthYear","Year",
               "Actor1Code","Actor1Name","Actor1CountryCode","Actor1KnownGroupCode","Actor1Type1Code",
               "Actor2Code","Actor2Name","Actor2CountryCode","Actor2KnownGroupCode","Actor2Type1Code",
               "IsRootEvent","EventCode","EventBaseCode","EventRootCode",
               "QuadClass","GoldsteinScale","NumMentions","NumSources","NumArticles","AvgTone",
               "Actor1Geo_FullName","Actor1Geo_CountryCode","Actor1Geo_ADM1Code","Actor1Geo_Lat","Actor1Geo_Long",
               "Actor2Geo_FullName","Actor2Geo_CountryCode","Actor2Geo_ADM1Code","Actor2Geo_Lat","Actor2Geo_Long",
               "ActionGeo_FullName","ActionGeo_CountryCode","ActionGeo_ADM1Code","ActionGeo_Lat","ActionGeo_Long",
               "SOURCEURL"]

In [11]:
#ubah path_clean sesuai tempat clean di lokal
path_clean = 'E:/digitalent-project/csv/2017-clean.csv'
data_gdelt = pd.read_csv(path_clean, delimiter = ',', encoding = "ISO-8859-1", names=header)
data_gdelt.sample(4)

,GlobalEventId,sqldate,MonthYear,Year,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1Type1Code,Actor2Code,...,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_Lat,Actor2Geo_Long,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,SOURCEURL
1046203,694904606,20171004,201710,2017,GOV,MINIST,NaN,NaN,GOV,NaN,...,NaN,NaN,NaN,NaN,Nigeria,NI,NI,10.0000,8.0000,http://naija247news.com/2017/10/03/25bln-contr...
334402,640705415,20170327,201703,2017,NaN,NaN,NaN,NaN,NaN,AUS,...,AS,AS02,-33.8833,151.2170,"Guangdong, Jilin, China",CH,CH05,42.7683,129.3360,http://www.abc.net.au/news/2017-03-27/ice-bust...
764068,672673361,20170713,201707,2017,USAGOV,UNITED STATES,USA,NaN,GOV,GOV,...,US,USDC,38.8951,-77.0364,"White House, District of Columbia, United States",US,USDC,38.8951,-77.0364,http://www.mysanantonio.com/news/crime/article...
848986,679245603,20170807,201708,2017,NaN,NaN,NaN,NaN,NaN,AFRGOV,...,SF,SF11,-34.0333,18.3500,"Hout Bay, Western Cape, South Africa",SF,SF11,-34.0333,18.3500,https://www.timeslive.co.za/news/south-africa/...


In [14]:
stop_words = set(stopwords.words('english'))

In [24]:
# contoh 1 URL
sample=data_gdelt['SOURCEURL'][0]
sample
sample_ID = data_gdelt['GlobalEventId'][0]

In [20]:
result = {}
for i in range(0,5):
     link = data_gdelt['SOURCEURL'][i]
     geventID = data_gdelt['GlobalEventId'][i]
     get_link(geventID,link)

http://nbc16.com/news/local/train-hits-and-kills-teen-boy-in-keizer
http://www.heraldtribune.com/news/20161231/lyons-frontier-is-worthy-suspect-in-fire-alarm-glitch
https://www.thenews.com.pk/print/175863-Official-figures-say-10-murdered-32-robbed-daily-in-Punjab-in-2016
http://www.news.com.au/national/nsw-act/new-years-eve-two-dead-another-critical-but-police-praise-sydneysiders/news-story/cb63d606b821dfa702ba10cb993f9d15
http://www.princegeorgecitizen.com/news/local-news/senior-vancouver-police-officer-charged-with-sex-crimes-and-breach-of-trust-1.5803598


In [91]:
res = {}

# Fungsi Scrapping

In [95]:
def scrap_article(gid, source):
    article = Article(source)
    article.download()
    article.parse()
    article.nlp()
    
    #jadiin lowercase
    txt = article.text.lower()
    #hapus \n \t dll
    txt_clean = ''.join([x for x in txt if x in string.ascii_letters + '\'- '])
    #jadikan kata perkata
    word_tokens = word_tokenize(txt_clean)
    #filter kata: yang disimpan cuma kata2 yang ga ada di stopword
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    print(type(filtered_sentence))
    #hitung kemunculan tiap kata
    aa = dict((Counter(filtered_sentence)).most_common(20))
    res[gid] = aa
    return aa
    

In [96]:
for x in range(2):
    # contoh 1 URL
    sample_=data_gdelt['SOURCEURL'][x]
    sample_ID_ = data_gdelt['GlobalEventId'][x]
    aaa = scrap_article(sample_ID_, sample_)
    print(aaa, type(aaa))

<class 'list'>
{'keizer': 6, 'train': 4, 'police': 4, 'ne': 3, 'marion': 3, 'county': 3, 'station': 2, 'tepper': 2, 'lane': 2, 'department': 2, 'say': 2, 'railroad': 2, 'time': 2, 'deceased': 2, 'office': 2, 'ore': 1, 'hit': 1, 'killed': 1, 'pedestrian': 1, 'near': 1} <class 'dict'>
<class 'list'>
{'frontier': 11, 'alarm': 10, 'fire': 8, 'one': 7, 'company': 7, 'sarasota': 4, 'building': 4, 'medical': 4, 'told': 4, 'department': 4, 'said': 4, 'many': 4, 'problem': 4, 'could': 3, 'back': 3, 'minutes': 3, 'two': 3, 'getting': 3, 'log': 3, 'contacted': 3} <class 'dict'>


In [60]:
print(type(aaa))

<class 'dict'>


In [97]:
res

{613248567: {'keizer': 6,
  'train': 4,
  'police': 4,
  'ne': 3,
  'marion': 3,
  'county': 3,
  'station': 2,
  'tepper': 2,
  'lane': 2,
  'department': 2,
  'say': 2,
  'railroad': 2,
  'time': 2,
  'deceased': 2,
  'office': 2,
  'ore': 1,
  'hit': 1,
  'killed': 1,
  'pedestrian': 1,
  'near': 1},
 613248634: {'frontier': 11,
  'alarm': 10,
  'fire': 8,
  'one': 7,
  'company': 7,
  'sarasota': 4,
  'building': 4,
  'medical': 4,
  'told': 4,
  'department': 4,
  'said': 4,
  'many': 4,
  'problem': 4,
  'could': 3,
  'back': 3,
  'minutes': 3,
  'two': 3,
  'getting': 3,
  'log': 3,
  'contacted': 3}}